<a href="https://colab.research.google.com/github/malikouda/notebooks/blob/main/marvel_champions_universal_prebuilt_changelog_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os, re
from collections import OrderedDict

import gdown
from google.colab import auth
from googleapiclient.discovery import build
from IPython.display import display
import pandas as pd


auth.authenticate_user()
files = OrderedDict()

gdrive = build('drive', 'v3').files()

# These are the archived decklists
# https://drive.google.com/drive/folders/1M2qAy6ddI7Doj7G1UcX4Me71tnPtwM9J
folder_id = "1M2qAy6ddI7Doj7G1UcX4Me71tnPtwM9J"

# Ordered by modified time
res = gdrive.list(q=f"'{folder_id}' in parents", orderBy='modifiedTime').execute()
for f in res['files']:
    files[f['id']] = f['name']

# This is the current list
# https://docs.google.com/spreadsheets/d/1uDnn-7Urtprf3awFV0cSUHseIqJQYCVF7ckineFVBP4/edit?usp=sharing
files['1uDnn-7Urtprf3awFV0cSUHseIqJQYCVF7ckineFVBP4'] = 'Current'

In [ ]:
!rm -rf decks; mkdir decks; ls

decks  sample_data


In [ ]:
releases = []

csv_url = "https://docs.google.com/spreadsheets/d/{0}/gviz/tq?tqx=out:csv&sheet={1}"
for id,name in files.items():
    name = re.sub(r'\W+', '', name)
    releases.append(name)
    gdown.download(url=csv_url.format(id, "Prebuilt Decks"), output=f'./decks/{name}', quiet=True, fuzzy=True)

In [ ]:
!ls /content/decks

AgentsofSHIELD	   Iceman		   Nova		    StarLordGamora
AgeofApocalypse    Ironheart		   Phoenix	    Valkyrie
AntMan		   JubileeJubilationLee    Psylocke	    Venom
BlackPantherShuri  MadTitansShadow	   Quicksilver	    Vision
Current		   Magneto		   Rogue	    WarMachine
Cyclops		   MutantGenesis	   ScarletWitch     Wasp
Drax		   Nebula		   SinisterMotives  WolverineLogan
GalaxysMostWanted  NeXtEvolution	   Spdr
Gambit		   NightcrawlerKurtWagner  SpiderHam


In [ ]:
def read_csv(f):
    df = pd.read_csv(f)
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    if all('(' not in col for col in df.columns):
        for col in df.columns:
            if col not in hero_alter_egos:
                raise Exception(f"{col} not in hero list")
            if hero_alter_egos[col] is None:
                continue
            df.rename(columns={col: f"{col} ({hero_alter_egos[col]})"}, inplace=True)
    df = df.iloc[1:]
    return df

In [ ]:
hero_alter_egos = {}

# Gets most all alter egos for heroes from most recent decklist
heroes = read_csv(f'/content/decks/{releases[-1]}').columns
for hero in heroes:
    if '(' in hero:
        groups = tuple(re.search(r'^(.+)\((.+)\)$', hero).groups())
        hero, alter_ego = groups
        hero = hero.strip()
        alter_ego = alter_ego.strip()
        if hero in hero_alter_egos:
            hero_alter_egos[hero].append(alter_ego)
        else:
            hero_alter_egos[hero] = [alter_ego]
    else:
        hero_alter_egos[hero] = None

# Creates a dictionary with "legacy" alter egos for older lists
# that didn't list alter egos. For example: Old decklists only
# listed 'Spider-Man', need to correct those to 'Spider-Man (
# Peter Parker)
for hero in hero_alter_egos.keys():
    ego_list = hero_alter_egos[hero]
    if ego_list is None:
        continue
    else:
        if len(ego_list) > 1:
            print(hero)
            print('[', '|'.join(ego_list), ']')
            print('using', ego_list[0])
            print()
        # Always use earliest alter-ego in any case
        hero_alter_egos[hero] = ego_list[0]

Spider-Man
[ Peter Parker|Miles Morales ]
using Peter Parker

Black Panther
[ T'Challa|Shuri ]
using T'Challa



In [ ]:
def compare_lists(old, new):
    entered = [i for i in new if i not in old]
    exited = [i for i in old if i not in new]
    return entered, exited

In [ ]:
class Style():
  RED = "\033[31m"
  GREEN = "\033[32m"
  BLUE = "\033[34m"
  RESET = "\033[0m"

style = Style()

def compare_release(r1, r2):
    df_old = read_csv(r1)
    df_new = read_csv(r2)

    print("\n# Heroes (old):", len(df_old.columns))
    print("# Heroes (new):", len(df_new.columns))

    for col in df_new.columns:
        new = df_new[col].tolist()

        if col not in df_old.columns:
            old = []
        else:
            old = df_old[col].tolist()

        entered, exited = compare_lists(old, new)

        if entered or exited:
            print()
            print(col)
            print('-'*50)
            for item in exited:
                print(f"{style.RED}-{item}{style.RESET}")
            for item in entered:
                print(f"{style.GREEN}+{item}{style.RESET}")
            print('-'*50)


In [ ]:
for i, name in enumerate(releases):
    print(f'[{i:>3n}]: {name}')

confirm = "m"

while confirm != "y":
    if confirm == "q":
        raise KeyboardInterrupt
    past = int(input("\nPick previous decklist: "))
    print(f'Selected "{style.BLUE}{releases[past]}{style.RESET}"')
    new = int(input("Pick new decklist: "))
    print(f'Selected "{style.BLUE}{releases[new]}{style.RESET}"')
    confirm = input("Confirm selections? (y/n/q): ")

past_release = f'/content/decks/{releases[past]}'
new_release = f'/content/decks/{releases[new]}'

compare_release(past_release, new_release)

[  0]: MadTitansShadow
[  1]: Nebula
[  2]: Venom
[  3]: Drax
[  4]: StarLordGamora
[  5]: GalaxysMostWanted
[  6]: ScarletWitch
[  7]: Quicksilver
[  8]: Wasp
[  9]: AntMan
[ 10]: WarMachine
[ 11]: Vision
[ 12]: Valkyrie
[ 13]: SinisterMotives
[ 14]: Nova
[ 15]: Ironheart
[ 16]: SpiderHam
[ 17]: Spdr
[ 18]: MutantGenesis
[ 19]: Cyclops
[ 20]: Phoenix
[ 21]: WolverineLogan
[ 22]: Gambit
[ 23]: Rogue
[ 24]: NeXtEvolution
[ 25]: Psylocke
[ 26]: AgeofApocalypse
[ 27]: Iceman
[ 28]: JubileeJubilationLee
[ 29]: NightcrawlerKurtWagner
[ 30]: Magneto
[ 31]: AgentsofSHIELD
[ 32]: BlackPantherShuri
[ 33]: Current

Pick previous decklist: 27
Selected "Iceman"
Pick new decklist: 33
Selected "Current"
Confirm selections? (y/n/q): y

# Heroes (old): 52
# Heroes (new): 59

Spider-Man (Peter Parker)
--------------------------------------------------
-Quake (Daisy Johnson)
-Monica Chang
-Surveillance Team
-Surveillance Team
-Surveillance Team
-Heroic Intuition
-Nick Fury
-First Aid
-First Aid
-Governm